# WEB SCRAPING DA TABELA DE ALIMENTOS 

    Usando os pacotes _requests_ e _scrapy_. 

In [ ]:
from scrapy import Selector
import requests
import pandas as pd

In [ ]:
# The basic URL
url = "http://www.tbca.net.br/base-dados/"

# A list of pages, the table is divided by 53 pages
paginas = []

# building the 53 pages urls by iteration
for i in range(1,54):
    pag = url + "composicao_alimentos.php?pagina=" + str(i)
    paginas.append(pag)
#paginas



In [ ]:
# getting the Codes (COD) of each product to use with the individual urls
links_seguir = []
for pagina in paginas:
    x = requests.get(pagina) # usando o pacote requests para pegar as urls RESPONSE
    html = x.text # com requests tranforma o html da pagina em string
    sel = Selector(text = html) #com scrapy usa a funcao Selector para receber o html em string
    urls_prods = sel.xpath('//tr/td[1]/a/@href').extract() # extrai o text da tag passada na xpath e faz uma lista
    links_seguir = links_seguir + urls_prods # concatena a lista de codigos em cada iteracao

links_completos = []
for link in  links_seguir:
    pags = url + link # link eh do tipo /int_composicao_alimentos.php?cod_produto=C0573F
    links_completos.append(pags)



    
#codigo pesado, gera uma lista de 5203 codigos

In [ ]:
# define a empty dictionary to be transformed in a Dataframe 
futuro_df = dict()

The **componente** function is gonna collect each variable from to its own xpath (caminho) and be put inside the dict **futuro_df**

In [ ]:

def componente(caminho, variavel): 
    # caminho eh uma string com xpath e nome eh o nome da variavel
    nome = []

    for i in links_completos:
        yy = requests.get(i)
        httt = yy.text #transforma cada link em texto
        sell = Selector(text = httt) # Selector seleciona cada texto
        linhas = sell.xpath(caminho).extract() # procura o caminho passado como xpath e extrai o texto dentro da tag
        # strip() tira os espaços do começo e do final
        nome = nome + linhas #add cada novo dado a lista
    
    nome = [x.strip() for x in nome]
    
    futuro_df[variavel] = nome
    return futuro_df

Each fuction call will take a very long time, because it's >5k observation each variable... just to inform!

In [ ]:
# codigos
componente('//h5[@id="overview"]/text()[1]', 'codigo')

# alimento
componente('//h5[@id="overview"]/text()[2]', 'alimento_pt')

#calorias
componente('//tbody/tr[2]/td[3]/text()', 'kcal')

#carboidratos
componente('//tbody/tr[4]/td[3]/text()', 'carboidratos')

# proteina
componente('//tbody/tr[6]/td[3]/text()', 'proteina')

# gordura
componente('//tbody/tr[7]/td[3]/text()', 'gordura')


Transforming the dictionary in a Dataframe

In [ ]:
df = pd.DataFrame.from_dict(futuro_df)
df.head(10)

And finally writing the collected data in a .csv file.

In [ ]:
arquivo_csv = df.to_csv('tbca.csv',index=False,sep=';')